In [23]:
import os
from PIL import Image
import pandas as pd
import shutil

In [2]:
labels = pd.read_csv("statues-train/statues_labels.csv")
labels.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Balashikha_0204.JPG,1296,864,lenin,683,198,709,230
1,New_Mexico_947.jpg,1404,936,other,619,205,648,234
2,Skopje_037.JPG,1128,752,other,457,34,509,97
3,Skopje_037.JPG,1128,752,other,414,267,480,346
4,Skopje_037.JPG,1128,752,other,715,287,789,349


In [4]:
class_dict = {
    "other" : 0,
    "lenin" : 1,
    "ataturk":2
}

In [6]:
unique_imgs = labels.filename.unique()

In [ ]:
for img in unique_imgs:
    anns = labels.loc[labels["filename"] == img]
    filename = img.split(".")[0]
    f = open(f"labels/{filename}.txt", "w+")
    for i, ann in anns.iterrows():    
        width, height = ann["width"], ann["height"]
        x1 = float(ann["xmin"])/ann["width"]
        y1 = float(ann["ymin"])/ann["height"]
        x2 = float(ann["xmax"])/ann["width"]
        y2 = float(ann["ymax"])/ann["height"]

        # w = x2 - x1
        # h = y2 - y1
        
        # x_center = x1 + w / 2.
        # y_center = y1 + h / 2.

        xmin = int(ann["xmin"])
        ymin = int(ann["ymin"])
        xmax = int(ann["xmax"])
        ymax = int(ann["ymax"])

        # w = x2 - x1
        # h = y2 - y1
        # x_center = min(xmin, xmax) + w / 2.
        # y_center = min(ymin, ymax) + h / 2.

        w = abs(xmin-xmax) / width
        h = abs(ymin-ymax) / height
        x_center = (min(xmin, xmax) / width) + w / 2.0
        y_center = (min(ymin, ymax) / height) + h / 2.0

        class_id = class_dict[ann["class"]]
        line = f"{class_id} {x_center} {y_center} {w} {h}\n"
        f.write(line)

In [21]:
## create image labels
folder_path = "/home/ubuntu/statues-train/"
train_annt = os.listdir("/home/ubuntu/yolov5/datasets/data/labels/train")
val_annt = os.listdir("/home/ubuntu/yolov5/datasets/data/labels/val")
total = 0
count = 0
for root, _, files in os.walk(folder_path):
    for file in files:
        if not file.lower().endswith(".jpg"):
            continue
        ann_filename = f"{os.path.splitext(file)[0]}.txt"
        if ann_filename in train_annt or ann_filename in val_annt:
            continue
        total += 1

for root, _, files in os.walk(folder_path):
    for file in files:
        if not file.lower().endswith(".jpg"):
            continue
        ann_filename = f"{os.path.splitext(file)[0]}.txt"
        if ann_filename in train_annt or ann_filename in val_annt:
            continue
        if count <= int(total * .8):
            path = os.path.join("/home/ubuntu/yolov5/datasets/data/labels/train", ann_filename)
            count += 1
        else:
            path = os.path.join("/home/ubuntu/yolov5/datasets/data/labels/val", ann_filename)
        with open(path, "w+") as fw:
            fw.write("")

In [28]:
image_dict = {}
for root, _, files in os.walk(folder_path):
    for file in files:
        if not file.lower().endswith(".jpg"):
            continue
        ann_filename = f"{os.path.splitext(file)[0]}.txt"
        image_dict[ann_filename] = os.path.join(root, file)

phases = ["train", "val"]
for phase in phases:
    for ann_file in os.listdir(f"/home/ubuntu/yolov5/datasets/data/labels/{phase}"):
        path = os.path.join(f"/home/ubuntu/yolov5/datasets/data/images/{phase}", os.path.basename(image_dict[ann_file]))
        os.makedirs(os.path.dirname(path), exist_ok=True)
        shutil.copy(image_dict[ann_file], path)
        # print(image_dict[ann_file], path)

In [29]:
for file in os.listdir("./"):
    shutil.rmtree()

SyntaxError: unexpected EOF while parsing (3311767521.py, line 1)

In [ ]:
s